In [ ]:
# default_exp model

In [ ]:
#export
import importlib
import copy
from functools import partial

import pytorch_lightning as pl
import torch
from torch.utils.data import DataLoader
from torch.optim import lr_scheduler
import torch.optim as optim
from adamp import AdamP

from defocus.utilities import get_metrics, get_GOPRO_lists
from defocus.losses import ContentLoss, AdversarialLoss, PerceptualLoss
from defocus.dataset import BaseDataset

In [ ]:
#export
class Model(pl.LightningModule):
    def __init__(self, hparams):
        super().__init__()
        self.hparams = hparams
        G_architecture = self.hparams.model.generator.architecture
        D_architecture = self.hparams.model.discriminator.architecture
        # TODO: maybe I shouldn't use importlib but do something else?
        self.G = importlib.import_module('defocus.architecture.' + G_architecture).Generator() 
        self.D = importlib.import_module('defocus.architecture.' + D_architecture).Discriminator() 
        # TODO: for consistency, there should be a self.P and all that stuff maybe?
        
#         self.content_loss = ContentLoss(name=self.hparams.model.loss.content_loss.name,
#                                         multiscale=self.hparams.model.loss.content_loss.multiscale,
#                                        )
#         self.adversarial_loss = AdversarialLoss(name=self.hparams.model.loss.adversarial_loss.name,
#                                                 multiscale=self.hparams.model.loss.adversarial_loss.multiscale,
#                                                )
#         self.perceptual_loss = PerceptualLoss(name=self.hparams.model.loss.perceptual_loss.name,
#                                               layer=self.hparams.model.loss.perceptual_loss.layer,
#                                               criterion=self.hparams.model.loss.perceptual_loss.criterion,
#                                               multiscale=self.hparams.model.loss.adversarial_loss.multiscale,
#                                              )
#         self.adversarial_weight = self.hparams.model.loss.adversarial_loss.weight
#         self.content_weight = self.hparams.model.loss.content_loss.weight
#         # TODO: perceptual weight is untested
#         self.perceptual_weight = self.hparams.model.loss.perceptual_loss.weight
        self.content_loss = ContentLoss(**vars(self.hparams.model.loss.content_loss))
        self.adversarial_loss = AdversarialLoss(**vars(self.hparams.model.loss.adversarial_loss))
        self.perceptual_loss = PerceptualLoss(**vars(self.hparams.model.loss.perceptual_loss))
        
    def training_step(self, batch, batch_idx, optimizer_idx=None):
        input_, target = batch
        output = self.G(input_)
        
        if optimizer_idx == 0:
            # TODO: remove commented lines after testing
#             loss_d = self.adversarial_weight * self.adversarial_loss.calculate_D_loss(self.D, output, target)
            loss_d = self.adversarial_loss.calculate_D_loss(self.D, output, target)
            # lightning's logging practices are ALWAYS changing, this was for 1.0.8    
            self.log('train_loss_d', loss_d)            
            return loss_d
        
        elif optimizer_idx == 1:
            # TODO: remove commented lines after testing
#             loss_content = self.content_weight * self.content_loss(output, target)
            loss_content = self.content_loss(output, target)
            loss_perceptual = self.perceptual_loss(output, target)
            # TODO: another level of commenting, remove and remove
#             # TODO: better one is the commented lines but for consistency, 
#             # I keep using the latter code. Once I am sure, uncomment these and delete the latter
#             # loss_adv = self.adversarial_weight * self.adversarial_loss.calculate_G_loss(self.D, output, target)
#             # loss_g = loss_content + loss_adv
#             loss_adv = self.adversarial_loss.calculate_G_loss(self.D, output, target)
#             loss_g = loss_content + self.adversarial_weight * loss_adv
            loss_adv = self.adversarial_loss.calculate_G_loss(self.D, output, target)
            loss_g = loss_content + loss_adv + loss_perceptual
            
            # lightning's logging practices are ALWAYS changing, this was for 1.0.8    
            self.log('train_loss_g', loss_g)
            self.log('train_loss_content', loss_content)
            self.log('train_loss_g_adv', loss_adv)
            self.log('train_loss_perceptual', loss_perceptual)
            return loss_g
        
    def validation_step(self, batch, batch_idx):
        # I had to do this. hooks didn't work. 
        # modifying on_validation_model_eval didn't work with ddp... 
        # this is as of lightning 1.0.8
        self.train()
        assert self.G.training, "Don't use the exp-avg batch norm stats, this is a GAN, it won't work."
        input_, target = batch
        output = self.G(input_)
        
        psnr, ssim = get_metrics(input_, output, target)    
        # lightning's logging best practices are ALWAYS changing, this was for 1.0.8 
        self.log('val_PSNR', torch.tensor(psnr).type_as(output), sync_dist=True)
        self.log('val_SSIM', torch.tensor(ssim).type_as(output), sync_dist=True)
    
    def configure_optimizers(self):
        # and finally setup Adam
        opt_G_func = get_optimizer(self.hparams.model.generator.optimizer)
        opt_D_func = get_optimizer(self.hparams.model.discriminator.optimizer)
        optimizer_G = opt_G_func(self.G.parameters())
        optimizer_D = opt_D_func(self.D.parameters())
        
        # and now actually setup the scheduler
        scheduler_G_func = get_scheduler(self.hparams.model.generator.scheduler)
        scheduler_D_func = get_scheduler(self.hparams.model.discriminator.scheduler)
        scheduler_G = scheduler_G_func(optimizer_G)
        scheduler_D = scheduler_D_func(optimizer_D) 
        
        return [optimizer_D, optimizer_G], [scheduler_D, scheduler_G]
    
    def forward(self, input_):
        return self.G(input_)
    
    def train_dataloader(self):
        # TODO: get_GOPRO_lists is not modular, change that thing and handle
        # the datasets, i.e. GOPRO or REDS or DIV2K or whatevet inside the get_thing()
        input_list, target_list = get_GOPRO_lists(root_folder=self.hparams.input.datapath.root_folder,
                                                  image_pair_list=self.hparams.input.datapath.image_pair_list
                                                 )
        train_set=BaseDataset(input_list=input_list,
                              target_list=target_list,
                              forward_process=self.hparams.input.training_forward_process_simulator,
                              augment=self.hparams.input.training_augmentations,
                              corrupt=self.hparams.input.training_corruptions,
                              normalize=self.hparams.input.training_normalization,
                              pyramid_levels=self.hparams.input.pyramid_levels,
                             )
        train_loader = DataLoader(train_set, 
                                  batch_size=self.hparams.training.batch_size, 
                                  num_workers=self.hparams.training.num_workers, 
                                  drop_last=True, 
                                  shuffle=True,
                                 )
        return train_loader
    
    def val_dataloader(self):
        # TODO: the get_DATASET_list() functions should be an input argument
        input_list, target_list = get_GOPRO_lists(root_folder=self.hparams.input.datapath.root_folder,
                                                  image_pair_list=self.hparams.input.datapath.val_image_pair_list
                                                 )
        val_set=BaseDataset(input_list=input_list,
                            target_list=target_list,
                            forward_process=self.hparams.input.validation_forward_process_simulator,
                            augment=self.hparams.input.validation_augmentations,
                            corrupt=self.hparams.input.validation_corruptions,
                            normalize=self.hparams.input.validation_normalization,
                            pyramid_levels=self.hparams.input.pyramid_levels,
                           )
        val_loader = DataLoader(val_set, 
                                batch_size=self.hparams.training.batch_size, 
                                num_workers=self.hparams.training.num_workers, 
                                drop_last=True,
                                shuffle=False,
                               )
        return val_loader
    
#     # lightning changes function arguments a lot as well, this is for version 1.0.8
#     def backward(self, loss, optimizer, optimizer_idx):
#         # I really don't get why I retained the graph after the D update?
#         # we feed a detached fake to D, so no gradients for G. which should mean that
#         # the backward should only destroy D's graph. 
#         # TODO: check this. go back to default backward and see what happens
#         # in vanilla pytorch: it works. but still, let's pay extra attention to zero_grads
#         # so as not to leak gradients anywhere
#         if optimizer_idx == 0:
#             loss.backward(retain_graph=True)
#         elif optimizer_idx == 1:
#             loss.backward()

# this also doesn't work as of lightning 1.0.8
#     def on_validation_model_eval(self):
#         # I don't like this way of doing things, feels non-lightning
#         # I made a comment here:
#         # https://github.com/PyTorchLightning/pytorch-lightning/issues/2551#issuecomment-742601083
#         self.train()
        
    
    def on_test_model_eval(self):
        # same thing as on_validation_model_eval
        self.train()
        
    def test_step(self, batch, batch_idx):
        assert self.G.training, "Don't use the exp-avg batch norm stats, this is a GAN, it won't work."
        input_, target = batch
        output = self.G(input_)
        
        psnr, ssim = get_metrics(input_, output, target)    
        # lightning's logging best practices are ALWAYS changing, this was for 1.0.8    
        self.log('test_PSNR', torch.tensor(psnr))
        self.log('test_SSIM', torch.tensor(ssim))
        return {'test_PSNR': torch.tensor(psnr), 'test_SSIM': torch.tensor(ssim)}
        
    def test_epoch_end(self, test_step_outputs):
        ssims = 0
        psnrs = 0
        for result in test_step_outputs:
            ssims += result['test_SSIM']
            psnrs += result['test_PSNR']
        length = len(test_step_outputs)
        print('test_SSIM is {} and test_PSNR is {}'.format(ssims/length, psnrs/length))

In [ ]:
#export
# this func is usually just to get Adam using the config.yaml parameters
def get_optimizer(args):
    kwargs = copy.deepcopy(vars(args))
    other_opts = {'AdamP': AdamP,
                 }
    name = kwargs.pop('name')
    if hasattr(optim, name):
        return partial(getattr(optim, name), **kwargs)
    else:
        return partial(other_opts[name], **kwargs)

In [ ]:
#export
# same thing, but to create a scheduler this time
def get_scheduler(args): 
    kwargs = copy.deepcopy(vars(args))
    other_schedulers = {'LinearDecay': LinearDecay,
                       }
    name = kwargs.pop('name')
    if hasattr(lr_scheduler, name):
        return partial(getattr(lr_scheduler, name), **kwargs)
    else:
        return partial(other_schedulers[name], **kwargs)

In [ ]:
#export 
# TODO: put this into some .py file
# but actually, don't use it, I don't see a reason for a new scheduler
# this is from DeblurGANv2 by the way
class LinearDecay(lr_scheduler._LRScheduler):
    """This class implements LinearDecay
    """

    def __init__(self, optimizer, num_epochs, start_epoch=0, min_lr=0, last_epoch=-1):
        """implements LinearDecay
        Parameters:
        ----------
        """
        self.num_epochs = num_epochs - start_epoch # I changed this # num_epochs
        self.start_epoch = start_epoch
        self.min_lr = min_lr
        super().__init__(optimizer, last_epoch)

    def get_lr(self):
        if self.last_epoch < self.start_epoch:
            return self.base_lrs
        return [base_lr - ((base_lr - self.min_lr) / self.num_epochs) * (self.last_epoch - self.start_epoch) for
                base_lr in self.base_lrs]

In [ ]:
from nbdev.export import *
notebook2script()

Converted 01_utilities.ipynb.
Converted 02_architecture_common.ipynb.
Converted 03_architecture_MSResNet.ipynb.
Converted 04_architecture_DeblurGANv2.ipynb.
Converted 05_blurring.ipynb.
Converted 06_dataset.ipynb.
Converted 07_losses.ipynb.
Converted 08_callbacks.ipynb.
Converted 09_model.ipynb.
Converted fuckit.ipynb.
